In [2]:
import base64
import requests
import os
import pandas as pd
from tqdm import tqdm  # Import the tqdm library for progress bar
import time
import re

# OpenAI API Key (replace with your own API key)
my_api_key = "your_api_key_here"

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Specify the folder path
folder_path = "/path/to/your/folder"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {my_api_key}"
}

# Function to extract number for sorting files numerically
def extract_number(file_name):
    match = re.search(r'(\d+)', file_name)
    return int(match.group(1)) if match else -1

results = []
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".png")], key=extract_number)  # Sort files by number

try:
    for filename in tqdm(file_list, desc="Processing images"):
        image_path = os.path.join(folder_path, filename)
        base64_image = encode_image(image_path)

        payload = {
            "model": "gpt-4o",
            "temperature": 0.0,
            "messages": [
                {
                    "role": "system",
                    "content": [
                        {
                            "type": "text",
                            "text": """
Based on the picture of the fictional character, evaluate their appearance. Rate how aggressive the character looks from 1 to 7, with 7 being the most aggressive. Please answer with numbers only.
"""
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                # "detail": "low"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 1
        }

        for attempt in range(5):  # Maximum 5 attempts
            try:
                response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
                response.raise_for_status()
                response_data = response.json()
                aggression_score = response_data.get('choices', [{}])[0].get('message', {}).get('content')
                results.append({'filename': filename, 'aggression_score': aggression_score})
                break  # Exit the loop if successful
            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}")
                if 'Connection error' in str(e):
                    print("Connection error detected. Waiting for 5 minutes before retrying...")
                    time.sleep(300)  # Wait for 5 minutes
                else:
                    time.sleep(1)  # Wait for 1 second for other errors
                if attempt == 4:
                    print("Max retries reached. Saving progress and exiting.")
                    raise e

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Convert results to DataFrame
    df_results = pd.DataFrame(results)
    # Save to Excel file
    df_results.to_excel("/path/to/save/gpt_sd_full_final.xlsx", index=False)
    print("Excel file has been saved successfully.")


Processing images:   1%|          | 52/5185 [04:20<7:56:09,  5.57s/it]

Request failed: 502 Server Error: Bad Gateway for url: https://api.openai.com/v1/chat/completions


Processing images:   5%|▍         | 249/5185 [20:20<8:02:35,  5.87s/it]

Request failed: 520 Server Error:  for url: https://api.openai.com/v1/chat/completions


Processing images:   7%|▋         | 350/5185 [28:29<6:19:13,  4.71s/it]

...

Request failed: 520 Server Error:  for url: https://api.openai.com/v1/chat/completions


Processing images:  99%|█████████▉| 5139/5185 [6:57:30<03:12,  4.17s/it]

Request failed: 520 Server Error:  for url: https://api.openai.com/v1/chat/completions


Processing images: 100%|██████████| 5185/5185 [7:01:15<00:00,  4.87s/it]

Excel file has been saved successfully.
